# Testing tensor contraction on a batch of matrices (large matrix).
<p>Program tests the parallel execution of a large number of matrix-matrix multiplications of the form</p>

$\Psi_{ik} \times \Psi^*_{ik}$

<p>of dimension ($2^k, 2^{n-k}$) where $2 <= k <= n/2$ and $\Psi_{ik}\in\mathbb{C}^{k\times n-k}$. 

Currently, the operating dimension is $n = 12$ but the goal is to reach $25$.

The experiment consists of a large number ($10^3 - 10^4$) of Monte Carlo simulations (MCS) each performing up to $10^4$ steps.
In each MCS in each step 2 matrix-matrix products of the matrices of the dimension $2^k \times 2^{n-k}$ are computed.

**<p>The aim of this notebook is to design and test a parallel algorithm to perform the above calculations.</p>**
    

## Parallelization strategy

The parallelisation can be divided into 3 levels:

**1<sup>st</sup> level** - Each MCS is performed independely of the other MCSs, thus all MCSs can be performed in parallel during one step (in one step two matrix-matrix multiplications are performed in each MCS)

**2<sup>nd</sup> level** - In each step of a MCS two matrix-matrix multiplications are performed independently and can be performed in parallel

**3<sup>rd</sup> level** - Parallel matrix-matrix product. For larger $n$ and $k$ close to $n/2$ the matrices become large and computation time increases. 


## Install required packages and import

In [1]:
import sys
print(sys.executable)
!which python
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install "dask[complete]"
!{sys.executable} -m pip install "graphviz"
!{sys.executable} -m pip install "scipy"
#sys.path.append('/home/jupiter/.local/bin')

/home/ddavidov/dask-virtenv/bin/python
/home/ddavidov/dask-virtenv/bin/python


In [2]:
import numpy as np
import scipy.special
import time
import dask.array as da

**Fix parameters $n$ and $k$, compute the matrix size and the number of matrix products (matrix pairs) that has to be computed**

In [3]:
nMonteCarlo = np.power(10,3)
nsteps = np.power(10,1)
n = 25
k = int(n/2)
num_rows = pow(2,k)
num_cols = pow(2,n-k)

In [4]:
print('Program parameters:')
print('Number of Monte Carlo simulations: ' + str(nMonteCarlo))
print('Number of steps per simulation: ' + str(nsteps))
print('Parameters n = ' + str(n) + ' k = ' + str(k))
print('Matrix size (2^k x 2^{n-k}) = ' + str(num_rows) + ' x ' + str(num_cols))

Program parameters:
Number of Monte Carlo simulations: 1000
Number of steps per simulation: 10
Parameters n = 25 k = 12
Matrix size (2^k x 2^{n-k}) = 4096 x 8192


**Generate a complex matrix of dimension $2^k \times 2^{n-k}$**

In [8]:
!export OMP_NUM_THREADS=1
!export MKL_NUM_THREADS=1
!export OPENBLAS_NUM_THREADS=1
A_real = da.random.random((num_rows, num_cols),chunks=(1024,1024))-0.5
A_imag = da.random.random((num_rows, num_cols),chunks=(1024,1024))-0.5
#A_real = da.random.random((num_rows, num_cols),chunks='auto')-0.5
#A_imag = da.random.random((num_rows, num_cols),chunks='auto')-0.5
A = A_real + A_imag*(0+1j)

In [9]:
A[0:num_rows,0:num_cols]

dask.array<add, shape=(4096, 8192), dtype=complex128, chunksize=(1024, 1024), chunktype=numpy.ndarray>

## Compute a matrix-matrix product - as regular np array

**Convert dask A to numpy A**

In [7]:
numpy_A = np.array(A)

**Compute a $A \times A^T$ - as a regular numpy array**

In [8]:
start = time.perf_counter()
numpy_A.dot(numpy_A.transpose())
numpy_A.dot(numpy_A.transpose())
end = time.perf_counter()
elapsed = end - start
print('Total time (numpy): ' + str(elapsed) + ' sec')

Total time (numpy): 4.801971063978272 sec


**Setup single-machine scheduler**

In [15]:
import dask
dask.config.set(scheduler='threads')

#!export OMP_NUM_THREADS=1
#!export MKL_NUM_THREADS=1
#!export OPENBLAS_NUM_THREADS=1

**Compute matrix product**

In [23]:
C1 = A.dot(A.transpose())
C2 = A.dot(A.transpose())

start = time.perf_counter()
#for iter in np.arange(num_matrices):
C1.compute()
C2.compute()

end = time.perf_counter()
elapsed = end - start
print('Total time (dask): ' + str(elapsed) + ' sec')

Total time (dask): 11.080421785009094 sec


**Setup distributed scheduler**

In [10]:
from dask.distributed import Client

client = Client(processes=False)

client

Client Scheduler: inproc://10.210.112.40/9428/1 Dashboard: http://10.210.112.40:8787/status,Cluster Workers: 1 Cores: 12 Memory: 16.60 GB


In [11]:
#!export OMP_NUM_THREADS=1
#!export MKL_NUM_THREADS=1
#!export OPENBLAS_NUM_THREADS=1

C1 = A.dot(A.transpose())
C2 = A.dot(A.transpose())

start = time.perf_counter()
#for iter in np.arange(num_matrices):
C1.compute()
C2.compute()

end = time.perf_counter()
elapsed = end - start
print('Total time (dask): ' + str(elapsed) + ' sec')

Total time (dask): 22.62793309101835 sec


In [32]:
client.close()